In [2]:
import pandas as pd

In [3]:
areas = ['City of London', 'Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster']

## Housing density

In [4]:
housing_density = pd.read_excel('data/housing density.xlsx', sheet_name= 'Number of dwellings')
housing_density = housing_density.drop(index=0)
for area_london in housing_density['Area name']:
    if area_london not in areas:
        housing_density = housing_density[housing_density['Area name'] != area_london]
housing_density.rename(columns={'Area name': 'Area'}, inplace=True)
housing_density.rename(columns={'ONS code': 'Code'}, inplace=True)

# TODO: actually compute the density --> waiting for mischa's data
housing_density

,Code,Area,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,E09000001,City of London,5009.0,5117.100000,5126.200000,5263.300000,5374.400000,5373.500000,5374.600000,5420.700000,...,5458.900000,5513.0,5531.000000,5566.000000,6003.000000,6229.000000,6306.000000,6313.000000,6451.000000,6506.000000
2,E09000002,Barking and Dagenham,68298.0,68525.700000,68837.400000,68899.100000,69260.800000,69528.500000,69835.200000,70550.900000,...,70946.300000,71079.0,71431.000000,71937.000000,72668.000000,73182.000000,73914.000000,74510.000000,74923.000000,75829.000000
3,E09000003,Barnet,130515.0,130800.600000,131883.200000,132855.800000,133902.400000,134941.000000,135428.600000,136680.200000,...,138616.400000,139346.0,141460.645482,142834.645482,143947.645482,145271.645482,146729.645482,148528.645482,150736.645482,152945.645482
4,E09000004,Bexley,91606.0,91907.600000,92223.200000,92949.800000,93220.400000,93433.000000,93745.600000,94078.200000,...,94798.400000,95037.0,95240.000000,95658.000000,96186.000000,96996.000000,96864.000000,97628.000000,97905.000000,98391.000000
5,E09000005,Brent,101427.0,102828.100000,103846.200000,104454.300000,105093.400000,106767.500000,107969.600000,109036.700000,...,111413.900000,112083.0,112643.000000,113305.000000,114039.000000,115598.000000,116649.000000,118013.000000,118707.000000,120448.000000
6,E09000006,Bromley,128717.0,128974.200000,129314.400000,129808.600000,130639.800000,131387.000000,132339.200000,133112.400000,...,134297.800000,135036.0,135612.000000,136304.000000,136453.000000,136864.000000,137564.000000,138422.000000,138976.000000,139684.000000
7,E09000007,Camden,93112.0,93884.100000,94460.200000,94841.300000,95563.400000,96350.500000,96997.600000,97516.700000,...,99138.900000,99828.0,100196.000000,100760.000000,101214.000000,101653.000000,102618.000000,103826.000000,104771.000000,105598.000000
8,E09000008,Croydon,141300.0,141423.600000,141819.200000,142176.800000,142566.400000,142961.000000,143713.600000,144896.200000,...,147249.400000,148099.0,148806.000000,149697.000000,150992.000000,152515.000000,154559.000000,157394.000000,159470.000000,161060.000000
9,E09000009,Ealing,120331.0,120589.700000,120922.400000,121428.100000,121909.800000,122743.500000,124111.200000,125530.900000,...,126901.300000,127187.0,127872.000000,128862.000000,129631.000000,130528.000000,131249.000000,132094.000000,133551.000000,135305.000000
10,E09000010,Enfield,112948.0,114130.900000,115273.800000,116337.700000,117055.600000,117613.500000,118667.400000,119946.300000,...,121243.100000,122042.0,122339.000000,122888.000000,123400.000000,123799.000000,124471.000000,125369.000000,125755.000000,126255.000000


## Crime

In [8]:
crime = pd.read_csv('data/crime.csv')
crime = crime.drop(crime.columns[[0,1]], axis=1)
crime = crime.reset_index(drop=True)
for col in crime.columns[1:]:
    if '04' in col:
        continue
    else:
        crime = crime.drop(col, axis=1)

crime = crime.groupby('LookUp_BoroughName').agg('sum').reset_index()

for area_london in crime['LookUp_BoroughName']:
    if area_london not in areas:
        crime = crime[crime['LookUp_BoroughName'] != area_london]

new_name = {}
for col in crime.columns[1:]:
    new_name[col] = col[:4]

crime.rename(columns={'LookUp_BoroughName': 'Area'}, inplace=True)
crime.rename(columns=new_name, inplace=True)
crime = crime.dropna()

# Adding missing Area Column
crime = pd.merge(housing_density[['Code', 'Area']],crime, on = 'Area')

# save clean data

crime = crime[['Code','Area','2011','2012','2013','2014','2015','2016','2017']]
crime = pd.melt(crime, id_vars=['Code','Area'], var_name='Year', value_name='Crime_rate')

crime.rename(columns = {'Crime_rate': 'Number_of_crimes' }, inplace= True)

crime.to_csv('clean_data/crime.csv',index=False)

crime

,Code,Area,Year,Number_of_crimes
0,E09000002,Barking and Dagenham,2011,1635
1,E09000003,Barnet,2011,2243
2,E09000004,Bexley,2011,982
3,E09000005,Brent,2011,2601
4,E09000006,Bromley,2011,1797
...,...,...,...,...
219,E09000029,Sutton,2017,953
220,E09000030,Tower Hamlets,2017,2727
221,E09000031,Waltham Forest,2017,1762
222,E09000032,Wandsworth,2017,1936


## Waste

In [9]:
waste = pd.read_excel('data/recycle.xlsx', sheet_name='Household Recycling Rates')
waste = waste.drop(index=0)
waste = waste.drop('Code', axis=1)
for area_london in waste['Area']:
    if area_london not in areas:
        waste = waste[waste['Area'] != area_london]
new_name = {}
for col in waste.columns[2:]:
    new_name[col] = col[:4]

waste.rename(columns={'New Code': 'Code'}, inplace=True)
waste.rename(columns=new_name).dropna()
waste.rename(columns=new_name, inplace=True)
waste = waste.dropna()

# save clean data

waste = waste[['Code','Area','2011','2012','2013','2014','2015','2016','2017']]
waste = pd.melt(waste, id_vars=['Code','Area'], var_name='Year', value_name='waste')

waste.rename(columns = {'waste': 'Waste_recycling_rates' }, inplace= True)

waste.to_csv('clean_data/waste.csv',index=False)

waste

,Code,Area,Year,Waste_recycling_rates
0,E09000001,City of London,2011,36.900000
1,E09000002,Barking and Dagenham,2011,30.000000
2,E09000003,Barnet,2011,33.600000
3,E09000004,Bexley,2011,53.500000
4,E09000005,Brent,2011,36.800000
...,...,...,...,...
226,E09000029,Sutton,2017,50.048815
227,E09000030,Tower Hamlets,2017,26.449413
228,E09000031,Waltham Forest,2017,32.481245
229,E09000032,Wandsworth,2017,22.085340
